In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy.stats import beta
from scipy import stats
np.random.seed(seed=42)
import math 
import sklearn.metrics
from sklearn.metrics import confusion_matrix, accuracy_score, r2_score
from sklearn.model_selection import train_test_split

In [2]:
import random
from random import randint

In [3]:
import re  

# Importing & Data Cleaning

In [4]:
pd.set_option('display.max_columns', 500)

In [5]:
accidents = pd.read_csv('~/Downloads/US_Accidents_June20.csv', delimiter = ',', header = 0, low_memory=False)

In [6]:
accidents.drop(['Source', 'End_Lat', 'End_Lng', 'Number', 'Weather_Timestamp', 'Airport_Code', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'], axis=1, inplace = True)

In [7]:
accidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3513617 entries, 0 to 3513616
Data columns (total 40 columns):
 #   Column             Dtype  
---  ------             -----  
 0   ID                 object 
 1   TMC                float64
 2   Severity           int64  
 3   Start_Time         object 
 4   End_Time           object 
 5   Start_Lat          float64
 6   Start_Lng          float64
 7   Distance(mi)       float64
 8   Description        object 
 9   Street             object 
 10  Side               object 
 11  City               object 
 12  County             object 
 13  State              object 
 14  Zipcode            object 
 15  Country            object 
 16  Timezone           object 
 17  Temperature(F)     float64
 18  Wind_Chill(F)      float64
 19  Humidity(%)        float64
 20  Pressure(in)       float64
 21  Visibility(mi)     float64
 22  Wind_Direction     object 
 23  Wind_Speed(mph)    float64
 24  Precipitation(in)  float64
 25  Weather_Condition 

In [8]:
accidents.isnull().sum()

ID                         0
TMC                  1034799
Severity                   0
Start_Time                 0
End_Time                   0
Start_Lat                  0
Start_Lng                  0
Distance(mi)               0
Description                1
Street                     0
Side                       0
City                     112
County                     0
State                      0
Zipcode                 1069
Country                    0
Timezone                3880
Temperature(F)         65732
Wind_Chill(F)        1868249
Humidity(%)            69687
Pressure(in)           55882
Visibility(mi)         75856
Wind_Direction         58874
Wind_Speed(mph)       454609
Precipitation(in)    2025874
Weather_Condition      76138
Amenity                    0
Bump                       0
Crossing                   0
Give_Way                   0
Junction                   0
No_Exit                    0
Railway                    0
Roundabout                 0
Station       

In [9]:
accidents.loc[accidents['Wind_Direction']=='Calm','Wind_Direction'] = 'CALM'
accidents.loc[(accidents['Wind_Direction']=='West')|(accidents['Wind_Direction']=='WSW')|(accidents['Wind_Direction']=='WNW'),'Wind_Direction'] = 'W'
accidents.loc[(accidents['Wind_Direction']=='South')|(accidents['Wind_Direction']=='SSW')|(accidents['Wind_Direction']=='SSE'),'Wind_Direction'] = 'S'
accidents.loc[(accidents['Wind_Direction']=='North')|(accidents['Wind_Direction']=='NNW')|(accidents['Wind_Direction']=='NNE'),'Wind_Direction'] = 'N'
accidents.loc[accidents['Wind_Direction']=='Variable','Wind_Direction'] = 'VAR'

In [10]:
# accidents['Clear'] = np.where(accidents['Weather_Condition'].str.contains('Clear', case=False, na = False), 1, 0)
# accidents['Cloud'] = np.where(accidents['Weather_Condition'].str.contains('Cloud|Overcast', case=False, na = False), 1, 0)
# accidents['Rain'] = np.where(accidents['Weather_Condition'].str.contains('Rain|storm', case=False, na = False), 1, 0)
# accidents['Heavy_Rain'] = np.where(accidents['Weather_Condition'].str.contains('Heavy Rain|Rain Shower|Heavy T-Storm|Heavy Thunderstorms', case=False, na = False), 1, 0)
# accidents['Snow'] = np.where(accidents['Weather_Condition'].str.contains('Snow|Sleet|Ice', case=False, na = False), 1, 0)
# accidents['Heavy_Snow'] = np.where(accidents['Weather_Condition'].str.contains('Heavy Snow|Heavy Sleet|Heavy Ice Pellets|Snow Showers|Squalls', case=False, na = False), 1, 0)
# accidents['Fog'] = np.where(accidents['Weather_Condition'].str.contains('Fog', case=False, na = False), 1, 0)

# # Assign NA to created weather features where 'Weather_Condition' is null.
# weather = ['Clear','Cloud','Rain','Heavy_Rain','Snow','Heavy_Snow','Fog']
# for i in weather:
#   accidents.loc[accidents['Weather_Condition'].isnull(),i] = accidents.loc[accidents['Weather_Condition'].isnull(),'Weather_Condition']

# accidents.loc[:,['Weather_Condition'] + weather]

# accidents = accidents.drop(['Weather_Condition'], axis=1)

In [11]:
accidents['Start_Time'] = pd.to_datetime(accidents['Start_Time'], errors='coerce')
accidents['End_Time'] = pd.to_datetime(accidents['End_Time'], errors='coerce')

In [12]:
accidents['Timezone'] = accidents['Timezone'].str.lstrip('US/')

In [13]:
accidents['Year'] = pd.DatetimeIndex(accidents['Start_Time']).year

In [14]:
accidents['Month'] = pd.DatetimeIndex(accidents['Start_Time']).month

In [15]:
import calendar

In [16]:
accidents['Month'] = accidents['Month'].apply(lambda x: calendar.month_abbr[x])

In [17]:
accidents['DayOfWeek'] = accidents['Start_Time'].apply(lambda x: x.weekday())

In [18]:
accidents['DayOfWeek'] = accidents['DayOfWeek'].apply(str)

In [19]:
accidents['DayOfWeek'] = accidents['DayOfWeek'].replace(['0','1','2', '3', '4', '5', '6'], ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun'])

In [20]:
accidents['Duration'] = accidents['End_Time'] - accidents['Start_Time']
accidents['Durination_InMinutes']=accidents['Duration']/np.timedelta64(1,'m')

In [21]:
accidents.round(1)

,ID,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,Side,City,County,State,Zipcode,Country,Timezone,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Year,Month,DayOfWeek,Duration,Durination_InMinutes
0,A-1,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.9,-84.1,0.0,Right lane blocked due to accident on I-70 Eas...,I-70 E,R,Dayton,Montgomery,OH,45424,US,Eastern,36.9,NaN,91.0,29.7,10.0,CALM,NaN,0.0,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,2016,Feb,Mon,05:14:00,314.0
1,A-2,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.9,-82.8,0.0,Accident on Brice Rd at Tussing Rd. Expect del...,Brice Rd,L,Reynoldsburg,Franklin,OH,43068-3402,US,Eastern,37.9,NaN,100.0,29.6,10.0,CALM,NaN,0.0,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,2016,Feb,Mon,00:30:00,30.0
2,A-3,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.1,-84.0,0.0,Accident on OH-32 State Route 32 Westbound at ...,State Route 32,R,Williamsburg,Clermont,OH,45176,US,Eastern,36.0,33.3,100.0,29.7,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,2016,Feb,Mon,00:30:00,30.0
3,A-4,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.7,-84.2,0.0,Accident on I-75 Southbound at Exits 52 52B US...,I-75 S,R,Dayton,Montgomery,OH,45417,US,Eastern,35.1,31.0,96.0,29.6,9.0,SW,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,2016,Feb,Mon,00:30:00,30.0
4,A-5,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.6,-84.2,0.0,Accident on McEwen Rd at OH-725 Miamisburg Cen...,Miamisburg Centerville Rd,R,Dayton,Montgomery,OH,45459,US,Eastern,36.0,33.3,89.0,29.6,6.0,SW,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,2016,Feb,Mon,00:30:00,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513612,A-3513776,NaN,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.0,-117.4,0.5,At Market St - Accident.,Pomona Fwy E,R,Riverside,Riverside,CA,92501,US,Pacific,86.0,86.0,40.0,28.9,10.0,W,13.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,2019,Aug,Fri,00:28:36,28.6
3513613,A-3513777,NaN,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.8,-117.1,0.3,At Camino Del Rio/Mission Center Rd - Accident.,I-8 W,R,San Diego,San Diego,CA,92108,US,Pacific,70.0,70.0,73.0,29.4,10.0,SW,6.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,2019,Aug,Fri,00:26:53,26.9
3513614,A-3513778,NaN,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.8,-117.8,0.6,At Glassell St/Grand Ave - Accident. in the ri...,Garden Grove Fwy,R,Orange,Orange,CA,92866,US,Pacific,73.0,73.0,64.0,29.7,10.0,S,10.0,0.0,Partly Cloudy,False,False,False,False,True,False,False,False,False,False,False,False,False,Day,2019,Aug,Fri,00:28:28,28.5
3513615,A-3513779,NaN,2,2019-08-23 19:00:21,2019-08-23 19:29:42,34.0,-118.4,0.8,At CA-90/Marina Fwy/Jefferson Blvd - Accident.,San Diego Fwy S,R,Culver City,Los Angeles,CA,90230,US,Pacific,71.0,71.0,81.0,29.6,10.0,SW,8.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,2019,Aug,Fri,00:29:21,29.4


In [22]:
accidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3513617 entries, 0 to 3513616
Data columns (total 45 columns):
 #   Column                Dtype          
---  ------                -----          
 0   ID                    object         
 1   TMC                   float64        
 2   Severity              int64          
 3   Start_Time            datetime64[ns] 
 4   End_Time              datetime64[ns] 
 5   Start_Lat             float64        
 6   Start_Lng             float64        
 7   Distance(mi)          float64        
 8   Description           object         
 9   Street                object         
 10  Side                  object         
 11  City                  object         
 12  County                object         
 13  State                 object         
 14  Zipcode               object         
 15  Country               object         
 16  Timezone              object         
 17  Temperature(F)        float64        
 18  Wind_Chill(F)         

In [28]:
accidents3 = accidents.sample(n = 500000)

In [29]:
accidents3.to_csv('Accidents3.csv')

In [23]:
for col, values in accidents.iteritems():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))
    print (values.unique())
    print ('\n')

ID: 3513617
['A-1' 'A-2' 'A-3' ... 'A-3513778' 'A-3513779' 'A-3513780']


TMC: 21
[201. 241. 247. 246. 341. 406. 245. 248. 200. 244. 203. 229. 222. 202.
 206. 343. 236. 239. 336. 339. 351.  nan]


Severity: 4
[3 2 1 4]


Start_Time: 3200042
['2016-02-08T05:46:00.000000000' '2016-02-08T06:07:59.000000000'
 '2016-02-08T06:49:27.000000000' ... '2019-08-23T19:11:30.000000000'
 '2019-08-23T19:00:21.000000000' '2019-08-23T18:52:06.000000000']


End_Time: 3246120
['2016-02-08T11:00:00.000000000' '2016-02-08T06:37:59.000000000'
 '2016-02-08T07:19:27.000000000' ... '2019-08-23T19:28:49.000000000'
 '2019-08-23T19:29:42.000000000' '2019-08-23T19:21:31.000000000']


Start_Lat: 1124695
[39.865147 39.928059 39.063148 ... 34.120911 33.943599 34.239104]


Start_Lng: 1113407
[ -84.058723  -82.831184  -84.032608 ... -118.416176 -117.14806
 -117.84779 ]


Distance(mi): 13476
[1.0000e-02 0.0000e+00 1.3200e+00 ... 7.4260e+00 3.6350e+01 1.6051e+01]


Description: 1780092
['Right lane blocked due to accident

In [24]:
accidents.to_csv('Accidents.csv')

In [25]:
accidents2 = pd.read_csv('Accidents.csv')

In [26]:
accidents2 = accidents.drop(columns='Unnamed: 0')

KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
accidents2.info()

# Visualizations - Accidents by Day, Month and Year

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt

In [ ]:
sns.set(rc={'figure.figsize': (16, 12)})

In [ ]:
#Number of Accidents Occuring by Day
sns.catplot(x = 'DayOfWeek', data=accidents2, kind='count')

In [ ]:
corr = accidents2.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [ ]:
#Number of Accidents Occuring by Month
sns.set(rc={'figure.figsize': (16, 12)})
sns.set_palette('GnBu_d')
sns.catplot(x = 'Month', data=accidents2, kind='count', order =accidents2.sort_values('Month').Month)

In [ ]:
# sns.set(rc={'figure.figsize': (16, 12)})
# sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
# ax = sns.boxplot(x = 'DayOfWeek', y = 'Duration', data = accidents, color = 'cyan')

In [ ]:
# sns.set(rc={'figure.figsize': (10, 6)})
# sns.catplot(x="OperatingSystem", y="TransactionAmt", hue="isFraud",
#             col="Timezone", aspect=.6,
#             kind="swarm", data=merged);

In [ ]:
#Accidents Occuring by the Hour 
accidents['TimeofAccident'] = pd.to_datetime(accidents2['Start_Time'], errors='coerce')
accidents['Hour'] = accidents2['Start_Time'] .dt.hour
accidents['Minute'] = accidents2['Start_Time'] .dt.minute
hours = [hour for hour, df in accidents2.groupby('Hour')]
plt.plot(hours, accidents2.groupby(['Hour'])['ID'].count())
plt.xticks(hours)
plt.xlabel('Time of Day by Hour')
plt.ylabel('Numer of Accidents')
plt.grid(True)
plt.show()

In [ ]:
#Accidents by Year
sns.set_context('talk')
sns.set_palette('magma')
a = sns.catplot(x='Year',data=accidents2,kind='count')
a.fig.suptitle('Accidents by Year ',y=1.03)
a.set(ylabel='Accident Count',xlabel='Year')
plt.show()

In [ ]:
sns.set(rc={'figure.figsize': (16, 12)})
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
ax = sns.boxplot(x = 'Severity', y = 'Year', data = accidents2, color = 'cyan')

In [ ]:
plt.figure(figsize=(20,12))
StatesAccidents = sns.countplot(y = 'State', data = accidents2, order = accidents2['State'].value_counts().index, palette='Blues')
plt.ylabel("State", labelpad=10, fontsize=10, weight = 'bold')
plt.xlabel('Count of Accidents', labelpad=10,fontsize=15, weight='bold')
plt.title('Accidents by State',fontsize=20, weight='bold')

In [ ]:
#Accident Severity and Duration
sns.set(rc={'figure.figsize': (12, 10)})
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
ax = sns.boxplot(x = 'Duration', y = 'Severity', data = accidents2)

In [ ]:
plt.figure(figsize=(20,12))
sns.barplot(surroundings['Accidents'],surroundings.index, palette = 'Blues_r')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel("Number of Traffic Accidents", labelpad = 10, fontsize=15,weight='bold')
plt.ylabel("Nearby Road Features", labelpad = 12,fontsize=15, weight='bold')
plt.title("Common Road Features and Traffic Accidents in California", fontsize=20,weight='bold')

In [ ]:
WeekendDF = accidents2[(accidents2['DayOfWeek'] == 'Saturday') | (df['DayOfWeek'] == 'Sunday')]
WeekendDF2 = WeekendDF.groupby('Hour').count()['ID']
WeekendDF2.plot(kind='bar', figsize=(10, 8))

# Visualizations - Severity and RoadRules/Types

In [30]:
accidents.Severity.value_counts()

2    2373210
3     998913
4     112320
1      29174
Name: Severity, dtype: int64

In [ ]:
#change the y-axis labels to the correct measure 
sns.set(rc={'figure.figsize': (6, 6)})
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Severity", data=accidents2, palette='autumn');

In [31]:
accidents.Sunrise_Sunset.value_counts()

Day      2593757
Night     919745
Name: Sunrise_Sunset, dtype: int64

In [ ]:
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Severity", data=accidents2, hue ='Sunrise_Sunset', palette='Accent');

In [32]:
accidents.Amenity.value_counts()

False    3471535
True       42082
Name: Amenity, dtype: int64

In [ ]:
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Severity", data=accidents2, hue ='Amenity', palette='Blues');

In [33]:
accidents.Bump.value_counts()

False    3513011
True         606
Name: Bump, dtype: int64

In [ ]:
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Severity", data=accidents2, hue ='Bump', palette='BrBG_r');

In [34]:
accidents.Crossing.value_counts()

False    3239091
True      274526
Name: Crossing, dtype: int64

In [ ]:
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Severity", data=accidents2, hue ='Crossing', palette='cividis');

In [43]:
accidents.Give_Way.value_counts()

False    3504053
True        9564
Name: Give_Way, dtype: int64

In [ ]:
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Severity", data=accidents2, hue ='Give_Way', palette='Wistia');

In [42]:
accidents.Junction.value_counts()

False    3229168
True      284449
Name: Junction, dtype: int64

In [ ]:
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Severity", data=accidents2, hue ='Junction', palette='copper');

In [41]:
accidents.No_Exit.value_counts()

False    3509233
True        4384
Name: No_Exit, dtype: int64

In [ ]:
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Severity", data=accidents2, hue ='No_Exit', palette='gnuplot');

In [40]:
accidents.Railway.value_counts()

False    3482442
True       31175
Name: Railway, dtype: int64

In [ ]:
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Severity", data=accidents2, hue ='Railway', palette='tab20b');

In [39]:
accidents.Roundabout.value_counts()

False    3513433
True         184
Name: Roundabout, dtype: int64

In [ ]:
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Severity", data=accidents2, hue ='Roundabout', palette='twilight');

In [38]:
accidents.Station.value_counts()

False    3443296
True       70321
Name: Station, dtype: int64

In [ ]:
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Severity", data=accidents2, hue ='Station', palette='ocean_r');

In [37]:
accidents.Stop.value_counts()

False    3461641
True       51976
Name: Stop, dtype: int64

In [ ]:
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Severity", data=accidents2, hue ='Stop', palette='pink');

In [36]:
accidents.Traffic_Signal.value_counts()

False    2889994
True      623623
Name: Traffic_Signal, dtype: int64

In [ ]:
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Severity", data=accidents2, hue ='Traffic_Signal', palette='Spectral');

In [35]:
accidents.Turning_Loop.value_counts()

False    3513617
Name: Turning_Loop, dtype: int64

In [ ]:
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Severity", data=accidents2, hue ='Turning_Loop', palette='autumn');

In [ ]:
# from wordcloud import WordCloud
# plt.style.use('seaborn')
# wrds1 = accidents["Description"].str.split("(").str[0].value_counts().keys()

# wc1 = WordCloud(scale=5,max_words=1000,colormap="rainbow",background_color="black").generate(" ".join(wrds1))
# plt.figure(figsize=(20,14))
# plt.imshow(wc1,interpolation="bilinear")
# plt.axis("off")
# plt.title("Patterned Words in Accident Description",color='b')
# plt.show()

In [ ]:
sns.set_context("poster", font_scale = 1.0, rc={"grid.linewidth": 1.0})
sns.pairplot(accidents2[['Severity', 'Distance(mi)', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)']], plot_kws = {"color": "green"}, height = 10)

In [ ]:
sns.set_context("poster", font_scale = 1.0, rc={"grid.linewidth": 1.0})
sns.pairplot(accidents2[['Pressure(in)', 'Severity', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']], plot_kws = {"color": "purple"}, height = 10)

In [ ]:
# sns.set(rc={'figure.figsize': (10, 6)})
# sns.catplot(x="OperatingSystem", y="TransactionAmt", hue="isFraud",
#             col="id_23", aspect=.6,
#             kind="swarm", data=merged);

In [ ]:
sns.set(rc={'figure.figsize': (6, 6)})
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Amenity", data=accidents2, palette='autumn');

In [ ]:
sns.set(rc={'figure.figsize': (6, 6)})
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Bump", data=accidents2, palette='autumn');

In [ ]:
sns.set(rc={'figure.figsize': (6, 6)})
sns.set_context("poster", font_scale = .75, rc={"grid.linewidth": 1.0})
sns.countplot(x="Amenity", data=accidents2, palette='autumn');

# Visualizations  - Environmental Impact